In [1]:
from pathlib import Path
import pandas as pd

In [2]:
import sys
import json
import argparse
from tqdm import tqdm

import evaluate
import pandas as pd
import numpy as np

from sectiontagger import SectionTagger

2023-04-15 20:09:42.642301: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-15 20:09:43.609809: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
print('Loading ROUGE, BERTScore, BLEURT from HuggingFace')
scorers = {
    'rouge': (
        evaluate.load('rouge'),
        {'use_aggregator': False},
        ['rouge1', 'rouge2', 'rougeL', 'rougeLsum'],
        ['rouge1', 'rouge2', 'rougeL', 'rougeLsum']
    ),
    'bert_scorer': (
        evaluate.load('bertscore'),
        {'model_type': 'microsoft/deberta-xlarge-mnli','batch_size':8},
        ['precision', 'recall', 'f1'],
        ['bertscore_precision', 'bertscore_recall', 'bertscore_f1']
    ),
    'bleurt': (
        evaluate.load('bleurt', config_name='BLEURT-20'),
        {},
        ['scores'],
        ['bleurt']
    ),
}

Loading ROUGE, BERTScore, BLEURT from HuggingFace
INFO:tensorflow:Reading checkpoint /root/.cache/huggingface/metrics/bleurt/BLEURT-20/downloads/extracted/cd1c38739d180ae53192201859a058307621534b704c20700072eca17d748c58/BLEURT-20.
INFO:tensorflow:Config file found, reading.
INFO:tensorflow:Will load checkpoint BLEURT-20
INFO:tensorflow:Loads full paths and checks that files exists.
INFO:tensorflow:... name:BLEURT-20
INFO:tensorflow:... bert_config_file:bert_config.json
INFO:tensorflow:... max_seq_length:512
INFO:tensorflow:... vocab_file:None
INFO:tensorflow:... do_lower_case:None
INFO:tensorflow:... sp_model:sent_piece
INFO:tensorflow:... dynamic_seq_length:True
INFO:tensorflow:Creating BLEURT scorer.
INFO:tensorflow:Creating SentencePiece tokenizer.
INFO:tensorflow:Creating SentencePiece tokenizer.
INFO:tensorflow:Will load model: /root/.cache/huggingface/metrics/bleurt/BLEURT-20/downloads/extracted/cd1c38739d180ae53192201859a058307621534b704c20700072eca17d748c58/BLEURT-20/sent_piece

2023-04-15 20:09:52.940998: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-04-15 20:09:52.942962: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


INFO:tensorflow:BLEURT initialized.


In [4]:
def filter_and_aggregate(obj, indices):

    agg_obj = {}
    for k, v in obj.items():
        agg_obj[k] = float(np.mean([v[i] for i in indices]))
    return agg_obj


In [5]:
baseline_path = Path.cwd().joinpath("baseline_model_summaries")

In [7]:
for f in baseline_path.glob("*.csv"):
    if "model_summaries" in f.stem:
        preds_df = pd.read_csv(f)
        model_prefix = "suryakiran786/5-fold-stratified-cv-"
        model_name_list = \
        [col for col in preds_df.columns if model_prefix in col]
        
        for model_name in model_name_list:
            
            preds_subset = preds_df[["ID","section_text",model_name]]
            
            references = preds_subset['section_text'].tolist()
            predictions = preds_subset[model_name].tolist()
            num_test = len(predictions)

            all_scores = {}
            for name, (scorer, kwargs, keys, save_keys) in tqdm(scorers.items(),desc="scorers"):
                scores = scorer.compute(references=references, predictions=predictions, **kwargs)
                for score_key, save_key in zip(keys, save_keys):
                    all_scores[save_key] = scores[score_key]

            cohorts = [
                        ('all', list(range(num_test))),
                    ]

            outputs = {k: filter_and_aggregate(all_scores, idxs) for (k, idxs) in cohorts}

            # ###### OUTPUT TO JSON FILE ########
            fn_out = f'{col.split("/")[-1]}.json'
            fn_out_path = baseline_path.joinpath(fn_out)
            print(f'Saving results to {fn_out}')
            with open(fn_out_path, 'w') as fd:
                json.dump(outputs, fd, indent=4)

scorers: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [10:27<00:00, 209.27s/it]
